In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import saspy
import optuna
from sklearn.preprocessing import LabelEncoder
import sklearn.metrics
# File system manangement
import os
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# Import Library
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

pd.reset_option('max_rows')
pd.reset_option('max_columns')

pd.set_option('max_rows',1000)
pd.set_option('max_columns',300)

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)


from scipy import stats
from scipy.stats import norm, skew, spearmanr #for some statistics

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, Ridge
from sklearn.linear_model import ElasticNetCV, LassoCV,  RidgeCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.svm import SVR
# import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression

In [ ]:
from lightgbm.sklearn import LGBMRegressor


In [ ]:
from __future__ import absolute_import, division, print_function

import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
from pycs.VariableAnalysis import *
from pycs.DataPreprocessing import *
from pycs.ModelAssessment import *

pd.reset_option('max_rows')
pd.reset_option('max_columns')

pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)

In [ ]:
def objective_param(varlist, params, num_boost_round=10000):
    hist_eval = dict()
    d_train = lgb.Dataset(train[varlist], label=train['GOOD'])
    d_valid = lgb.Dataset(valid[varlist], label=valid['GOOD'])

    model = lgb.train(params, d_train, num_boost_round=num_boost_round, valid_sets=[d_train, d_valid],
                      early_stopping_rounds=50, verbose_eval=None, evals_result=hist_eval)
#     return 2*hist_eval['valid_1']['auc'][-1]-1
    return 2*hist_eval['valid_1']['auc'][model.best_iteration-1]-1      #edit: 2022-11-28

def varselect_greedy(feature_list,params, initial_list=[],
                     method='forward', slentry=0.05, slstay=0.05, verbose=True, num_boost_round=10000):

    if method == 'forward':
        included = initial_list
        excluded = feature_list
        select = list(set(feature_list)-set(initial_list))
        forward = len(excluded) > 0
        curr_obj = 0
        obj_step = []
        selected_feature = []

        while (forward):
            obj_list = pd.Series(index=excluded, dtype='float64')

            for new_column in tqdm(excluded, desc='Forward Selection: '):
                curr_varlist = included + [new_column]
                obj_list[new_column] = objective_param(curr_varlist, params, num_boost_round=num_boost_round)

            best_obj = obj_list.max()
            best_feature = excluded[obj_list.argmax()]

            if ((best_obj-curr_obj) > slentry):
                included = included + [best_feature]
                obj_step += [best_obj]
                selected_feature += [best_feature]
                excluded = list(set(excluded)-set([best_feature]))
                curr_obj = best_obj
                if verbose==True:
                    print('Added  {:32} with Gini {:.2%}'.format(
                        best_feature, best_obj))
            else:
                forward = False
            if len(excluded) == 0:
                forward = False

        select_step = pd.DataFrame(zip(selected_feature, obj_step), columns=[
                                   'Entered Feature', 'Gini'])

    if method == 'backward':
        included = feature_list
        excluded = list(set(feature_list)-set(initial_list))
        backward = len(excluded) > 0
        curr_obj = objective(y, df)
        print('All var entered with Gini {:.2%}'.format(curr_obj))
        obj_step = [curr_obj]
        selected_feature = [np.nan]

        while (backward):
            obj_list = pd.Series(index=excluded, dtype='float64')

            for new_column in tqdm(excluded, desc='Backward Selection: '):
                curr_varlist = list(set(excluded)-set([new_column]))
                obj_list[new_column] = objective_param(curr_varlist)

            best_obj = obj_list.max()
            best_feature = excluded[obj_list.argmax()]

            if ((curr_obj-best_obj) < slstay):
                included = list(set(included) - set([best_feature]))
                obj_step += [best_obj]
                selected_feature += [best_feature]
                excluded = list(set(excluded)-set([best_feature]))
                curr_obj = best_obj
                if verbose==True:
                    print('Removed  {:32} with Gini {:.2%}'.format(
                        best_feature, best_obj))
            else:
                backward = False
            if len(excluded) == 0:
                backward = False

        select_step = pd.DataFrame(zip(selected_feature, obj_step), columns=[
                                   'Removed Feature', 'Gini'])

    if method == 'stepwise':
        included = initial_list
        excluded = list(set(feature_list)-set(initial_list))
        forward = len(excluded) > 0
        backward = len(included) > 0
        curr_obj = 0
        obj_step = []
        selected_feature = []

        while (forward or backward):
            # forward step
            obj_list = pd.Series(index=excluded, dtype='float64')

            for new_column in tqdm(excluded, desc='Forward Selection: '):
                curr_varlist = included + [new_column]
                obj_list[new_column] = objective_param(curr_varlist)

            best_obj = obj_list.max()
            best_feature = excluded[obj_list.argmax()]

            if ((best_obj-curr_obj) > slentry):
                included = included + [best_feature]
                obj_step += [best_obj]
                selected_feature += [best_feature]
                excluded = list(set(excluded)-set([best_feature]))
                curr_obj = best_obj
                if verbose==True:
                    print('Added  {:32} with Gini {:.2%}'.format(
                        best_feature, best_obj))
            else:
                forward = False
            if len(excluded) == 0:
                forward = False

            # backward step
            select = list(set(included)-set(initial_list))
            if len(select) == 1:
                continue
            obj_list = pd.Series(index=select, dtype='float64')

            for new_column in tqdm(select, desc='Backward Selection: '):
                curr_varlist = list(set(included)-set([new_column]))
                obj_list[new_column] = objective_param(curr_varlist)

            best_obj = obj_list.max()
            best_feature = included[obj_list.argmax()]

            if ((curr_obj-best_obj) < slstay):
                included = list(set(included) - set([best_feature]))
                obj_step += [best_obj]
                selected_feature += [best_feature]
                excluded = list(set(excluded)-set([best_feature]))
                curr_obj = best_obj
                if verbose==True:
                    print('Removed  {:32} with Gini {:.2%}'.format(
                        best_feature, best_obj))
            else:
                backward = False
            if len(excluded) == 0:
                backward = False

        select_step = pd.DataFrame(zip(selected_feature, obj_step), columns=[
                                   'Entered Feature', 'Gini'])

    return select_step, included

In [ ]:
def trans_value_charact(df_train, var, target, mapping=True):
    df = df_train.groupby(var).agg(Total=(target, 'count'), Bad=(target, 'sum'))
    df = df.rename_axis('Label').reset_index()
    df['Percent']=100*df['Total']/df_train.shape[0]
    df['Bad rate']=100*df['Bad']/df['Total']
    df = df.rename_axis('Group').reset_index()
    
    if df['Total'].sum()< df_train.shape[0]:
        df = df.append(pd.Series([df.shape[0],
                                  np.nan, 
                                  df_train[var].isnull().sum(),
                                  df_train[df_train[var].isnull()]['BAD'].sum(),
                                  100*df_train[var].isnull().sum()/df_train.shape[0],
                                  100*df_train[df_train[var].isnull()]['BAD'].sum()/df_train[var].isnull().sum()
                                 ],index=df.columns), 
                       ignore_index=True)
        
    
    df['Group'] = df['Group']+1
    if mapping==True:
        print(dict(zip(df['Label'],df['Group'])))
        display(df.style.background_gradient(cmap='Greens',subset='Bad rate',axis=0))
    return df

def add_value(df_train, df_valid, df_oot, varlist, target):
    for var in varlist:
        #train
        
        df_raw = trans_value_charact(df_train, var,target,mapping=False)
        df_raw['Group'] = df_raw['Label'].map(globals()[var])
        
        df_train['GRP_'+var]=df_train[var].map(globals()[var])
        df = trans_value_charact(df_train, 'GRP_'+var,target,mapping=False)
        globals()['GRP_'+var] = dict(zip(df['Label'],df['Bad rate']))
        df_train['GRP_'+var]=df_train['GRP_'+var].map(globals()['GRP_'+var]).astype('float')
        df_raw['Value'] = df_raw['Group'].map(globals()['GRP_'+var]).astype('float')
        
        print('GRP_'+var+"="+str(dict(zip(df_raw['Label'].fillna('np.nan'),df_raw['Value']))).replace("'np.nan'",'np.nan'))
                          
        #validate
        df_valid['GRP_'+var]=df_valid[var].map(globals()[var])
        df_valid['GRP_'+var]=df_valid['GRP_'+var].map(globals()['GRP_'+var]).astype('float')
        
        #out-of-time
        df_oot['GRP_'+var]=df_oot[var].map(globals()[var])
        df_oot['GRP_'+var]=df_oot['GRP_'+var].map(globals()['GRP_'+var]).astype('float')

    return df_train, df_valid, df_oot

In [ ]:
def bin_score(df, var, num=20):
    """
    Thường dùng trong Nrunbook trong trường hợp biến chưa có format, dùng function để xác định điểm chia
    """
    
    gr = df[[var]].assign(Label=lambda x: pd.qcut(x[var], num, duplicates='drop'))
    summary = gr.groupby('Label')[var].agg(Observations=('count'))
    summary = summary.rename_axis('Label').reset_index()
    summary['Label'] = summary['Label'].astype(str)

    #----------Edit Label of score------------
    UB=summary['Label'].apply(lambda x: float(x[x.find(',')+1 : len(x)-1]) if x!='Missing' else np.nan)
    UB[UB.idxmax()]=np.inf
    bin=list(map(float,['-inf'] + np.unique(np.array(UB)).tolist()))
    print(str(bin).replace('inf','np.inf'))
    return(bin) 

In [ ]:
def Nrunbook(df, target, var, bins=None, num=20):    
    """
    df: File tồn tại predict và target
    var: tên biến predict
    bins=None: Mặc định là runbook chưa có format. Nếu runbook đã có format có thể khai báo bins và dùng bins=bin_range
    num=20: Mặc định chia predict thành 20 nhóm nếu runbook chưa có format
    """
    if bins==None:    
        bins=bin_score(df, var,num)
    else:
        bins=bins
        
    df=df.assign(Band=lambda x: pd.cut(x[var],bins=bins,
                                           labels=['({},{}]'.format(bins[i],bins[i+1]) for i in range(len(bins)-1)],
                                           right=True))
    if target == 'GOOD':
        summary = df.groupby('Band').agg(Observations=(target, 'count'), Bad=('BAD', 'sum'))
    else:
        summary = df.groupby('Band').agg(Observations=(target, 'count'), Bad=(target, 'sum'))
        
    summary = summary.rename_axis('Band').reset_index()
    summary['Band'] = summary['Band'].astype(str)
    summary = summary.reset_index()
    summary['Band'] = summary['index'].apply(lambda x: str(x+1).zfill(2)) + '. '+summary['Band']
    summary = summary.drop(['index'], axis=1)
    
    
    #--------------- Missing ----------------
    if df[var].isnull().sum() >0:
        summary=summary.append(pd.Series(['Missing', df[var].isnull().sum(), df.BAD[df[var].isnull()].sum()],
                                         index=summary.columns), ignore_index=True)
    
    #--------------- Score distribution ----------------
    summary=summary.assign(Good=lambda x: x.Observations - x.Bad,
                           PerBadinBand=lambda x: round(100*x.Bad/x.Observations,2),
                           PerBadinTotalBad=lambda x: round(100*x.Bad/x.Bad.sum(),2),
                           PerGoodinTotalGood=lambda x: round(100*x.Good/x.Good.sum(),2),
                           PerCumBadinTotalBad=lambda x: round(100*x.Bad.cumsum()/x.Bad.sum(),2),
                           PerCumGoodinTotalGood=lambda x: round(100*x.Good.cumsum()/x.Good.sum(),2),
                           GoodPerBad=lambda x: round(x.Good/x.Bad,2)
                          ).rename(columns={'PerBadinBand':'%Bad in Band', 
                                            'PerBadinTotalBad':'%Bad/TotalBad',
                                            'PerGoodinTotalGood':'%Good/TotalGood',
                                            'PerCumBadinTotalBad':'%CumBad/TotalBad',
                                            'PerCumGoodinTotalGood':'%CumGood/TotalGood',
                                            'GoodPerBad':'Good/Bad'})
    #-----------Display------------
    display(summary.style.hide_index())
#     display(summary)

    #----------Plot------------
    ax = summary.loc[:,['%Bad/TotalBad', '%Good/TotalGood']].plot(kind="bar", stacked=False, color=('#1f77b4', '#d62728'))
    ax2 = ax.twinx()
    ax2.plot(np.arange(summary.shape[0]),summary['%Bad in Band'].values,linestyle='-',marker='o',linewidth=2.0,color='#2ca02c')
    plt.grid('off')
    plt.ylabel('Percents')
    plt.title('Score distribution')
    plt.show()

    #-----------GINI------------
    fpr_gb, tpr_gb, _ = roc_curve(df[target],df['Score'])
    train_roc_auc=roc_auc_score(df[target],df['Score']).round(5)
    plt.figure()
    plt.plot(fpr_gb, tpr_gb, label='GBM Baseline train (area = {:0.2f}' %train_roc_auc)
    plt.plot([0,1], [0,1], 'r')
    plt.title('Gini coeficient ({:0.2f}%)'.format(2*train_roc_auc*100-100))
    plt.show()
    return summary

In [ ]:
from bokeh.transform import dodge
from bokeh.models import LinearAxis, Range1d, ColumnDataSource, NumberFormatter, NumeralTickFormatter, Select, TableColumn, DataTable, BoxAnnotation, Tabs, Panel 
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.layouts import column, row
from bokeh.plotting import figure, output_file, show
from bokeh.io import  show, output_notebook
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

def model_stab(train, outtime, sample=None, nbin=20, bins=None):
    if  bins is None:
    # 1. Calculate cut point
        _, bins = pd.qcut(train, q=nbin, retbins=True, duplicates='drop')
        bins[0], bins[-1] = -np.inf, np.inf
    else:
        bins = bins
        
    # 2. Merge into 1 data
    if sample is not None:
        df = pd.concat([pd.DataFrame({'Pred': train, 'Sample': 0})] +
                       [pd.DataFrame({'Pred': outtime, 'Sample': sample})])
    else:
        df = pd.concat([pd.DataFrame(
            {'Pred': train, 'Sample': 0})]+[pd.DataFrame({'Pred': outtime, 'Sample': 1})])

    # 3. Summary by cutpoint
    df['Bin'] = pd.cut(df['Pred'], bins=bins)
    df['Bin'] = df['Bin'].astype(str)

    df_summary = df.groupby(['Sample', 'Bin']).agg(
        Count=('Pred', 'count')).reset_index()
    df_total = df.groupby(['Sample']).agg(
        Total=('Pred', 'count')).reset_index()

    # 4. Join data
    df_summary = df_summary.merge(df_total, on='Sample', how='left')
    df0 = df_summary[df_summary['Sample'] == 0][['Bin', 'Count', 'Total']]
    df_final = pd.DataFrame([(i, j) for i in list(sample.unique())
                            for j in df0['Bin'].to_list()], columns=['Sample', 'Bin'])
    df_final['Sample'] = df_final['Sample'].astype(int)
    df_final = df_final.merge(df0, on=['Bin'], how='left')
    df_final = df_final.merge(df_summary, on=['Sample', 'Bin'], how='left')

    # 5. PSI calculation
    df_final = df_final.assign(Pct_x=lambda x: x.Count_x/x.Total_x,
                               Pct_y=lambda x: x.Count_y/x.Total_y,
                               woe=lambda x: np.where(x.Count_x*x.Count_y == 0,
                                                      np.log(
                                                          ((x.Count_x+0.5)/x.Total_x)/((x.Count_y+0.5)/x.Total_y)),
                                                      np.log(((x.Count_x+0.0)/x.Total_x)/((x.Count_y+0.0)/x.Total_y))),
                               psi=lambda x: x.woe*(x.Count_x/x.Total_x-x.Count_y/x.Total_y))
    df_final['Bin'] = df_final['Bin'].astype(str)
    df_psi = df_final.groupby(['Sample']).agg(PSI=('psi', 'sum')).reset_index()
    samples = list(df_psi['Sample'])
    psis = list(df_psi['PSI'])

    # 6. Reporting
    init = samples[0]
    select = Select(title="Select sample:", width=200, value=str(
        init), options=[str(i) for i in samples])
    axis = [str(i) for i in range(len(df0))]
    psi = psis[0]
    # 6.1. Data source
    columns = [
        TableColumn(field="Bin", title="Bin"),
        TableColumn(field="Count_x", title="Count Dev",
                    formatter=NumberFormatter(format='0,0')),
        TableColumn(field="Pct_x", title="Percent Dev",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="Count_y", title="Count Rec",
                    formatter=NumberFormatter(format='0,0')),
        TableColumn(field="Pct_y", title="Percent Rec",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="psi", title="PSI",
                    formatter=NumberFormatter(format='00.00%'))
    ]
    df_source = ColumnDataSource(data=df_final[df_final['Sample'] == init])
    df_source.data['x'] = axis
    table = DataTable(source=df_source, columns=columns,
                      width=900, height=200, editable=False)
    ## 6.2. Figrure

    fig_dist = figure(plot_height=300, plot_width=900,
                      toolbar_location='right', tools="box_select, reset, save",
                      y_axis_label='Probability density function',
                      x_range=axis, x_axis_label='Scoreband',
                      title='Distribution Comparison(PSI= {:0.2%})'.format(psi))
    fig_dist.x_range.factors = axis
    fig_dist.yaxis.formatter = NumeralTickFormatter(format='00.00%')
    fig_dist.vbar(x=dodge('x', -0.20, range=fig_dist.x_range), top='Pct_x', width=0.3,  source=df_source,
                  color="#e84d60", legend_label='Percent Dev')
    fig_dist.vbar(x=dodge('x', +0.20, range=fig_dist.x_range), top='Pct_y', width=0.3,  source=df_source,
                  color="#718dbf", legend_label='Percent Rec')
    fig_dist.y_range.start = 0
    fig_dist.xgrid.grid_line_color = None
    fig_dist.title.align = "center"
    fig_dist.legend.location = 'top_left'

    # Figure: Dynamic PSI

    psi_select = [np.nan]*len(samples)
    psi_select[0] = psis[0]

    df_dn = ColumnDataSource(data=dict(x=[str(i) for i in samples],
                                       psi=psis,
                                       select=list(psi_select)))

    fig_dn = figure(plot_height=300, plot_width=900,
                    toolbar_location='right', tools="tap, reset, save",
                    y_range=(0, max(0.4, max(df_psi['PSI']))), y_axis_label='PSI',
                    x_range=[str(i) for i in samples], x_axis_label='Sample',
                    title='PSI Coefficient Dynamics (PSI = {:0.2%})'.format(psis[0]))

    fig_dn.add_layout(BoxAnnotation(bottom=0, top=0.1,
                      fill_alpha=0.5, fill_color='green'))
    fig_dn.add_layout(BoxAnnotation(bottom=0.1, top=0.2,
                      fill_alpha=0.5, fill_color='yellow'))
    fig_dn.add_layout(BoxAnnotation(bottom=0.2, top=0.4,
                      fill_alpha=0.5, fill_color='red'))
    fig_dn.line('x', 'psi', source=df_dn)
    fig_dn.circle('x', 'psi', source=df_dn, fill_color="white", size=5)
    fig_dn.circle('x', 'select', source=df_dn, fill_color="red", size=5)

    fig_dn.xgrid.grid_line_color = None
    fig_dn.title.align = "center"

    # Bokeh show

    def update_select(attrname, old, new):
        selected = int(select.value)
        idx = samples.index(selected)
        fig_dist.title.text = 'Distribution Comparison(PSI= {:0.2%})'.format(
            psis[idx])
        fig_dn.title.text = 'PSI Coefficient Dynamics (PSI = {:0.2%})'.format(
            psis[idx])

        psi_select = [np.nan]*len(samples)
        psi_select[idx] = psis[idx]

        df_dn.data = dict(x=[str(i) for i in samples],
                          psi=psis, select=list(psi_select))

        df_plot = df_final[df_final['Sample'] == selected].reset_index()
        df_source.data = df_plot
        df_source.data['x'] = df_plot.index.astype(str).to_list()

    def modify_doc(doc):
        doc.add_root(row(layout, width=900))

    select.on_change('value', update_select)
    report_data = Panel(child=column(fig_dn), title="Dynamic Result")
    report_fig = Panel(child=column(fig_dist, table), title="PSI Chart")

    layout = column(select, Tabs(tabs=[report_data, report_fig]))

    output_notebook()
    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

    display(df_psi.style.hide_index())

    distr_table = df_final[df_final['Sample'] == df_final['Sample'].max()].drop(['Sample'],axis=1)
    
    distr_table = distr_table[['Bin','Count_x','Count_y','Pct_x','Pct_y','psi']].rename(
        columns={'Count_x':'Count Dev',
                 'Pct_x':'Percent Dev',
                 'Count_y':'Count Rec',
                 'Pct_y':'Percent Rec', 
                 'psi':'PSI'})
    
    display(distr_table.style.hide_index().format({'Percent Dev': '{:.2%}', 
                                                   'Percent Rec': '{:.2%}', 
                                                   'PSI': '{:.2%}'}))
    
#     return distr_table

In [ ]:
def model_dcrm(actual, predict, sample=None, nbin=20, bins=None):
    if sample is not None:
        df = pd.DataFrame({'Actual': actual, 'Predict': predict,
                          'Sample': sample}, dtype=np.float64)
    else:
        df = pd.DataFrame(
            {'Actual': actual, 'Predict': predict, 'Sample': 0}, dtype=np.float64)
    df['Sample'] = df['Sample'].astype(int)
    if bins is None:
        df['Label'] = df.groupby(['Sample'])['Predict'].transform(lambda x: pd.qcut(x, nbin, duplicates='drop'))
    else:
        df['Label'] = df.groupby(['Sample'])['Predict'].transform(lambda x: pd.cut(x, bins=bins, right=True))
        
    df_grp = df.groupby(['Sample', 'Label']).agg(
        Total=('Actual', 'count'), Good=('Actual', 'sum')).reset_index()
    df_grp['Bad'] = df_grp['Total']-df_grp['Good']
    df_grp[['CumBad', 'CumGood']] = df_grp.groupby(
        ['Sample'])[['Bad', 'Good']].cumsum()

    df_by = df_grp.groupby('Sample')[['Total', 'Good', 'Bad']].sum()
    df_by['LabelFirst'] = df_grp.groupby(['Sample'])['Label'].first()
    df_by['LabelLast'] = df_grp.groupby(['Sample'])['Label'].last()
    df_grp = df_grp.merge(df_by, on=['Sample'], how='left')
    df_grp = df_grp.assign(
        Badrate=lambda x: x.Bad_x/x.Total_x,
        Goodrate=lambda x: x.Good_x/x.Total_x,
        PctBad=lambda x: x.Bad_x/x.Bad_y,
        PctGood=lambda x: x.Good_x/x.Good_y,
        CumPctBad=lambda x: x.CumBad/x.Bad_y,
        CumPctGood=lambda x: x.CumGood/x.Good_y,
        Odds=lambda x: np.where(x.Bad_x ==0, x.Good_x/(x.Bad_x+1),x.Good_x/x.Bad_x),
        Label=lambda x: x.Label.astype(str),
        comma=lambda x: x.Label.str.find(",").astype(int)
    ).rename(columns={'Total_x': 'Total', 'Good_x': 'Good', 'Bad_x': 'Bad'})\
        .drop(['LabelFirst', 'LabelLast', 'Total_y', 'Bad_y', 'Good_y', 'comma'], axis=1)

    columns = [
        TableColumn(field="Label", title="Label"),
        TableColumn(field="Total", title="Total",
                    formatter=NumberFormatter(format='0,0')),
        TableColumn(field="Bad", title="Bad",
                    formatter=NumberFormatter(format='0,0')),
        TableColumn(field="Good", title="Good",
                    formatter=NumberFormatter(format='0,0')),
        TableColumn(field="Badrate", title="Bad Rate",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="Goodrate", title="Good Rate",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="PctBad", title="Pdf Bad",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="PctGood", title="Pdf Good",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="CumPctBad", title="Cdf Bad",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="CumPctGood", title="Cdf Good",
                    formatter=NumberFormatter(format='00.00%')),
        TableColumn(field="Odds", title="Odds",
                    formatter=NumberFormatter(format='0,0.000'))
    ]

    samples = list(df_grp['Sample'].unique())
    ginis = [2*roc_auc_score(df.loc[df['Sample'] == i, 'Actual'],
                             df.loc[df['Sample'] == i, 'Predict'])-1 for i in samples]
    kss = [(df_grp.loc[df_grp['Sample'] == i, 'CumPctBad'] -
            df_grp.loc[df_grp['Sample'] == i, 'CumPctGood']).abs().max() for i in samples]
    init = samples[0]

    # Select sample

    select = Select(title="Select sample:", width=200, value=str(
        init), options=[str(i) for i in samples])
    df_plot = df_grp[df_grp['Sample'] == init]
    df_source = ColumnDataSource(data=df_plot)

    axis = df_plot.index.astype(str).to_list()
    df_source.data['x'] = axis

    table = DataTable(source=df_source, columns=columns,
                      width=900, height=200, editable=False)

    gini_select = [np.nan]*len(samples)
    gini_select[0] = ginis[0]
    df_dn = ColumnDataSource(data=dict(x=[str(i) for i in samples],
                                       gini=ginis,
                                       select=list(gini_select)))

    # Figure: Good/Bad Distribution

    fig_dist = figure(plot_height=350, plot_width=450,
                      toolbar_location='right', tools="box_select, reset, save",
                      y_axis_label='Percent in band',
                      x_range=axis, x_axis_label='Scoreband',
                      title='Good/Bad distribution by Scoreband')
    fig_dist.x_range.factors = axis
    fig_dist.yaxis.formatter = NumeralTickFormatter(format='00.00%')
    fig_dist.vbar(x=dodge('x', -0.20, range=fig_dist.x_range), top='PctGood', width=0.3,  source=df_source,
                  color="#e84d60", legend_label='Pdf Good')
    fig_dist.vbar(x=dodge('x', +0.20, range=fig_dist.x_range), top='PctBad', width=0.3,  source=df_source,
                  color="#718dbf", legend_label='Pdf Bad')
    
    
    fig_dist.y_range = Range1d(0, max(df_plot['PctBad'].max(),df_plot['PctGood'].max())+0.05)
    fig_dist.extra_y_ranges = {"y2_range": Range1d(0, df_plot['Odds'].max()+5)}
    fig_dist.add_layout(LinearAxis(y_range_name="y2_range",
                        axis_label="Good/Bad"), 'right')
    fig_dist.line('x', 'Odds', source=df_source, y_range_name="y2_range")
    fig_dist.circle('x', 'Odds', source=df_source, y_range_name="y2_range",
                         legend_label='Good/Bad', fill_color="white", size=5)

    fig_dist.y_range.start = 0
    fig_dist.xgrid.grid_line_color = None
    fig_dist.title.align = "center"
    fig_dist.add_layout(fig_dist.legend[0], 'below')
#     fig_dist.legend.location = 'top_left'
    fig_dist.legend.label_text_font_size = '8pt'

    
    # Figure: Gini Chart

    fig_gini = figure(plot_height=300, plot_width=450,
                      toolbar_location='right', tools="tap, reset, save",
                      y_range=(0, 1),
                      y_axis_label='%Cum Bad',
                      x_range=(0, 1),
                      x_axis_label='%Cum Good',
                      title='Gini Coefficient ({:0.2%})'.format(ginis[0]))

    fig_gini.line("CumPctBad", "CumPctGood", source=df_source,
                  color="#718dbf", legend_label='ROC curve')
    fig_gini.line("CumPctBad", "CumPctBad", source=df_source,
                  color="#e84d60")

    fig_gini.y_range.start = 0
    fig_gini.xgrid.grid_line_color = None
    fig_gini.title.align = "center"
    fig_gini.legend.location = 'bottom_right'

    # Figure: K-S Chart
    fig_ks = figure(plot_height=300, plot_width=450,
                    toolbar_location='right', tools="tap, reset, save",
                    y_range=(0, 1), y_axis_label='Percent in band',
                    x_range=axis, x_axis_label='Scoreband',
                    title='K-S Coefficient ({:0.2%})'.format(kss[0]))
    fig_ks.line("x", "CumPctGood", source=df_source,
                color="#e84d60", legend_label='Cdf Good')
    fig_ks.line("x", "CumPctBad", source=df_source,
                color="#718dbf", legend_label='Cdf Bad')
    fig_ks.y_range.start = 0
    fig_ks.xgrid.grid_line_color = None
    fig_ks.title.align = "center"
    fig_ks.legend.location = 'top_left'
    fig_ks.legend.label_text_font_size = '8pt'
    
    
    # Figure: Dynamic Gini
    fig_dn = figure(plot_height=300, plot_width=900,
                    toolbar_location='right', tools="tap, reset, save",
                    y_range=(0, 1), y_axis_label='Gini',
                    x_range=[str(i) for i in samples], x_axis_label='Sample',
                    title='Gini Coefficient Dynamics (Gini = {:0.2%})'.format(ginis[0]))

    fig_dn.add_layout(BoxAnnotation(bottom=0, top=0.4,
                      fill_alpha=0.5, fill_color='red'))
    fig_dn.add_layout(BoxAnnotation(bottom=0.4, top=0.6,
                      fill_alpha=0.5, fill_color='yellow'))
    fig_dn.add_layout(BoxAnnotation(bottom=0.6, top=1.0,
                      fill_alpha=0.5, fill_color='green'))
    fig_dn.line('x', 'gini', source=df_dn)
    fig_dn.circle('x', 'gini', source=df_dn, fill_color="white", size=5)
    fig_dn.circle('x', 'select', source=df_dn, fill_color="red", size=5)

    fig_dn.xgrid.grid_line_color = None
    fig_dn.title.align = "center"
    fig_dn.legend.label_text_font_size = '8pt'

    # ---------------------

    def update_select(attrname, old, new):
        selected = int(select.value)
        idx = samples.index(selected)
        fig_gini.title.text = 'Gini Coefficient ({:0.2%})'.format(ginis[idx])
        fig_ks.title.text = 'K-S Coefficient ({:0.2%})'.format(kss[idx])
        fig_dn.title.text = 'Gini Coefficient Dynamics (Gini = {:0.2%})'.format(
            ginis[idx])

        gini_select = [np.nan]*len(samples)
        gini_select[idx] = ginis[idx]

        df_dn.data = dict(x=[str(i) for i in samples],
                          gini=ginis, select=list(gini_select))

        df_plot = df_grp[df_grp['Sample'] == selected].reset_index()
        df_source.data = df_plot
        df_source.data['x'] = df_plot.index.astype(str).to_list()

    report_data = Panel(child=column(table, fig_dn), title="Dynamic Result")
    report_fig = Panel(child=column(
        row(fig_gini, fig_ks), fig_dist), title="ROC Chart")

    
    def modify_doc(doc):
        doc.add_root(row(layout, width=900))

    select.on_change('value', update_select)
    layout = column(select, Tabs(tabs=[report_fig, report_data]))

    output_notebook()
    handler = FunctionHandler(modify_doc)
    app = Application(handler)
    show(app)

    gini = pd.DataFrame(zip(samples, ginis, kss), columns=['Sample', 'Gini', 'K-S'])
    display(gini.style.hide_index())
    
    distr_table = df_plot.drop(['Sample'],axis=1).assign(Percent=lambda x: x.Total/x.Total.sum())
    distr_table = distr_table[['Label','Total','Percent','Good','Bad','Badrate',
                               'Goodrate','PctBad','PctGood', 'CumPctBad','CumPctGood','Odds']].rename(
        columns={'Label':'Band',
                 'Total':'#Obs',
                 'Percent':'%Obs',
                 'Badrate':'Bad rate',
                 'Goodrate':'Good rate',
                 'PctBad':'%Bad',
                 'PctGood':'%Good',
                 'CumPctBad':'%Cum Bad',
                 'CumPctGood':'%Cum Good'})
    
    display(distr_table.style.hide_index().format({'%Obs': '{:.2%}', 
                                                   'Bad rate': '{:.2%}', 
                                                   'Good rate': '{:.2%}', 
                                                   '%Bad': '{:.2%}', 
                                                   '%Good': '{:.2%}', 
                                                   '%Cum Bad': '{:.2%}', 
                                                   '%Cum Good': '{:.2%}', 
                                                   'Odds': '{:.3}'}))
#     return df_plot, gini


In [ ]:
def cal_fine(df):
    df2 = df.groupby('Group final').agg(Total=('Total', 'sum'), Bad=('Bad', 'sum'))
    df2=df2.reset_index()
    df2=df2.reset_index().rename(columns={'index':'Group'})
    df2['Label'] = ['Group '+str(int(i)) for i in df2['Group final']]
    df2['Percent'] = df2['Total']/df2['Total'].sum()
    df2['Bad rate']=df2['Bad']/df2['Total']
    return df2[['Group','Label','Total','Bad','Percent','Bad rate','Group final']]

class varbin2:
    
    def __init__(self, data, var, target):
        self.data=data
        self.var=var
        self.target=target
        
        # 1. Binning
        df_coarse = self.cal_coarse(data=self.data, var=self.var, target=self.target)
        df_coarse['Group final']=df_coarse['Group']
        
        # 2. Result
        self.df_coarse=df_coarse
        self.df_fine = cal_fine(self.df_coarse)
        self.df_backup = self.df_coarse.copy()
        
        
        # 3. Reporting
        ## 3.1. Data Source
        
        self.data_coarse = ColumnDataSource()
        self.data_fine = ColumnDataSource()
        
        ## 3.2. Figures
        
        ### 3.2.1. Coarse Figures
        axis = [str(i) for i in range(len(df_coarse))]
        
        self.fig_coarse = figure(plot_height=350, plot_width=450,
                          toolbar_location='right', tools="box_select, reset, save",
                          y_axis_label='Percent in group',
                          x_range=axis, x_axis_label=' ',
                          title=str(self.var) + ' (Coarse Classing)')
        self.fig_coarse.x_range.factors = axis
        self.fig_coarse.yaxis.formatter = NumeralTickFormatter(format='00.00%')
        self.fig_coarse.vbar(x=dodge('x', -0.20, range=self.fig_coarse.x_range), top='Percent', width=0.3,  
                             source=self.data_coarse,
                             color="#718dbf", legend_label='%Obs')    

        self.fig_coarse.y_range = Range1d(0, df_coarse['Percent'].max()+0.05)
        self.fig_coarse.extra_y_ranges = {"y2_range": Range1d(0, min(df_coarse['Bad rate'].max()+0.05,1))}
        self.fig_coarse.yaxis.formatter = NumeralTickFormatter(format='00.00%')

        self.fig_coarse.add_layout(LinearAxis(y_range_name="y2_range",axis_label="Transform value"), 'right')
        self.fig_coarse.line('x', 'Bad rate', source=self.data_coarse, y_range_name="y2_range", color='#2ca02c')
        self.fig_coarse.circle('x', 'Bad rate', source=self.data_coarse, y_range_name="y2_range",
                             legend_label='Transform value', fill_color="white", size=4)

        self.fig_coarse.y_range.start = 0
        self.fig_coarse.xgrid.grid_line_color = None
        self.fig_coarse.title.align = "center"
        self.fig_coarse.add_layout(self.fig_coarse.legend[0], 'below')
        self.fig_coarse.legend.label_text_font_size = '8pt'

        
        ### 3.2.1. Fine Figures
        self.fig_fine = figure(plot_height=350, plot_width=450,
                          toolbar_location='right', tools="box_select, reset, save",
                          y_axis_label='Percent in group',
                          x_range=axis, x_axis_label=' ',
                          title=str(self.var) + ' (Fine Classing)')
        self.fig_fine.x_range.factors = axis
        self.fig_fine.yaxis.formatter = NumeralTickFormatter(format='00.00%')
        self.fig_fine.vbar(x=dodge('x', -0.20, range=self.fig_fine.x_range), top='Percent', width=0.3,  
                           source=self.data_fine,
                           color="#718dbf", legend_label='%Obs')    

        self.fig_fine.y_range = Range1d(0, self.df_fine['Percent'].max()+0.05)
        self.fig_fine.extra_y_ranges = {"y2_range": Range1d(0, min(self.df_fine['Bad rate'].max()+0.05,1))}
        self.fig_fine.yaxis.formatter = NumeralTickFormatter(format='00.00%')

        self.fig_fine.add_layout(LinearAxis(y_range_name="y2_range",axis_label="Transform value"), 'right')
        self.fig_fine.line('x', 'Bad rate', source=self.data_fine, y_range_name="y2_range", color='#2ca02c')
        self.fig_fine.circle('x', 'Bad rate', source=self.data_fine, y_range_name="y2_range",
                             legend_label='Transform value', fill_color="white", size=4)

        self.fig_fine.y_range.start = 0
        self.fig_fine.xgrid.grid_line_color = None
        self.fig_fine.title.align = "center"
        self.fig_fine.add_layout(self.fig_fine.legend[0], 'below')
        self.fig_fine.legend.label_text_font_size = '8pt'
        
        ## 3.3. Buttons
        self.btn_manual = Button(label="Manual Bin", button_type="success", width=150)
        self.format = TextInput(title="Format", width=900)
        
        ## 3.4. Data Column
        columns = [
            TableColumn(field="Label", title="Range", width=50),
            TableColumn(field="Group final", title="Group"),
            TableColumn(field="Total", title="Total", formatter=NumberFormatter(format='0,0')), 
            TableColumn(field="Bad", title="Bad", formatter=NumberFormatter(format='0,0')),
            TableColumn(field="Percent", title="Percent", formatter=NumberFormatter(format='00.00%')),
            TableColumn(field="Bad rate", title="Bad rate", formatter=NumberFormatter(format='00.00%'))
        ]

        self.print_coarse = DataTable(
            source=self.data_coarse,
            columns=columns,
            width=900, height=200, editable=True,
        )
        
        self.print_fine = DataTable(
            source=self.data_fine,
            columns=columns,
            width=900, height=200, editable=False,
        )
        
        self.update()

        
    def cal_fine(self, df):
        df['Group final'] = df['Group final'].astype(int)
        df = df.sort_values(by=['Group final'])
        df2 = df.groupby('Group final').agg(Total=('Total', 'sum'), Bad=('Bad', 'sum'))
        df2=df2.reset_index()
        df2=df2.reset_index().rename(columns={'index':'Group'})
        
        df2['Label'] = ['Group '+str(int(i)) for i in df2['Group final']]
        df2['Percent'] = df2['Total']/df2['Total'].sum()
        df2['Bad rate']=df2['Bad']/df2['Total']
        return df2[['Group','Label','Total','Bad','Percent','Bad rate','Group final']]
    

    def cal_coarse(self, data, var, target):
        df = data.groupby(var).agg(Total=(target, 'count'), Bad=(target, 'sum'))
        df = df.rename_axis('Label').reset_index()
        df['Percent']=df['Total']/data.shape[0]
        df['Bad rate']=df['Bad']/df['Total']
        df = df.rename_axis('Group').reset_index()

        if df['Total'].sum()< data.shape[0]:
            df = df.append(pd.Series([df.shape[0],
                                      np.nan, 
                                      data[var].isnull().sum(),
                                      data[data[var].isnull()][target].sum(),
                                      data[var].isnull().sum()/data.shape[0],
                                      data[data[var].isnull()][target].sum()/data[var].isnull().sum()
                                     ],index=df.columns), 
                           ignore_index=True)

        return df
    
#     def update_cut_coarse(self, attrname, old, new):
#         self.df_coarse['Group final'] = list(self.cut_coarse)       
        
#         # 4. Result
#         self.df_coarse=df_coarse
#         self.df_fine = self.cal_fine(self.df_coarse)
#         self.df_backup = self.df_coarse.copy()
        
#         # 3. Reporting
#         ## 3.1. Data Source
        
#         self.fig_coarse.extra_y_ranges = {"y2": Range1d(0, (df_coarse['Bad rate'].max()))}
#         self.fig_fine.extra_y_ranges = {"y2": Range1d(0, (df_coarse['Bad rate'].max()))}
#         self.update()
            
    def update_data(self, attrname, old, new):
        self.df_backup['Group final']=new['Group final']
        
    def update_bin(self):
        self.df_coarse=self.df_backup
        self.df_fine=self.cal_fine(self.df_backup[['Total','Bad','Group final']])
        
        self.update()
          
        
    def update(self, update_coarse_all=True, update_btn_save=True):
        report_cols=['Group','Label','Total','Bad','Percent','Bad rate','Group final']
        
        #Coarse Data
        x_axis = list(zip(self.df_coarse['Group final'].astype(str), self.df_coarse['Group'].astype(str)))
        self.fig_coarse.title.text = str(self.var)+' (Coarse Classing)'
        self.fig_coarse.x_range.factors = x_axis
        if update_coarse_all:
            self.data_coarse.data = self.df_coarse[report_cols]
        else:
            self.data_coarse.data['Group final']=list(self.df_coarse['Group final'])
        self.data_coarse.data['x']=x_axis
        
        #Fine Data
        x_axis = list(self.df_fine['Group final'].astype(str))
        self.fig_fine.title.text = str(self.var)+' (Fine Classing)'
        self.fig_fine.x_range.factors = x_axis
        self.data_fine.data = self.df_fine[report_cols]
        self.data_fine.data['x']=x_axis
        
        #Format
        self.data_new = self.df_backup[['Label','Group final']].copy()
        self.data_new['Group final'] = self.data_new['Group final'].astype(int)
        
        self.data_new=pd.merge(self.data_new, 
                               self.df_fine[['Group final','Bad rate']].rename(columns={'Bad rate':'Value'}), 
                               on=['Group final'],
                               how='left')
        self.data_new['Value'] = round(self.data_new['Value']*100,5)
        
        self.format.value = 'GRP_'+str(self.var)+"="+str(dict(zip(self.data_new['Label'].fillna('np.nan'),
                                                            self.data_new['Value']))).replace("'np.nan'",'np.nan')
        
    def report(self):
        #1. Elements
        self.data_coarse.on_change('data', self.update_data)
        self.btn_manual.on_click(self.update_bin)
        
        # 2. Report
        report_coarse=Panel(child=column(self.print_coarse, self.fig_coarse), title="Coarse Binning")
        report_fine=Panel(child=column(self.print_fine, self.fig_fine), title="Fine Binning")

        layout = column(row(self.format), row(self.btn_manual), Tabs(tabs=[report_coarse, report_fine]))
        
        def modify_doc(doc):
            doc.add_root(row(layout, width=900))

        output_notebook()
        handler = FunctionHandler(modify_doc)
        app = Application(handler)
        show(app)